In [1]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
from dask.dot import dot_graph
import itertools
import logging
import netCDF4
import numpy as np
import dask.array as da
from dask import delayed
import time
from dask.distributed import Client
from urllib import request
from multiprocessing import Pool

client = Client('scheduler:8786')

download_location = '/temp'
data_url = 'http://nasanex.s3.amazonaws.com'
max_download_attempts = 5

all_models = ['ACCESS1-0',  'BNU-ESM', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM-CHEM', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M', 'bcc-csm1-1', 'inmcm4']
# all_models = ['ACCESS1-0', 'BNU-ESM'] 
all_vars = ['tasmax', 'pr']
all_years = {
     # 'historical': list(range(1971, 1976))
    'historical': list(range(1971, 2001))
}

def get_dataset_url(variable, scenario, model, year, prefix = data_url):
    prefix_filename = '/'.join([prefix, 'NEX-GDDP', 'BCSD', scenario, 'day', 'atmos', variable, 'r1i1p1', 'v1.0'])
    filename = '_'.join([variable, 'day', 'BCSD', scenario, 'r1i1p1', model, str(year) + '.nc'])
    url = '/'.join([prefix_filename, filename])
    return url

def get_context(year, **kwargs):
    variables = [kwargs.get('variable')] if kwargs.get('variable') else all_vars
    scenarios = ['historical']
    models = [kwargs.get('model')] if kwargs.get('model') else all_models
    outlist = []
    combinations = list(itertools.product(variables, scenarios, models))
    result = list(map(lambda comb: [ *comb, year ], combinations))
    return result

def get_year_ensemble(year, variable = 'tasmax'):
    context = get_context(year, variable = variable)
    datasets = list(map(lambda x: str(get_dataset_url(*x)), context))
    return datasets

def download_file(url):
    print("url: " + url)
    attempts = 0
    success = False
    filename = ""
    while attempts < max_download_attempts and not success:
        time.sleep(2 ** attempts)
        filename = '/'.join([download_location, str(url.split('/')[-1])])
        print("Downloading file at " + filename)
        u = request.urlopen(url)
        f = open(filename, 'wb')
        f.write(u.read())
        f.close()
        success = True
        break
    return filename

def download_file_list(url_list):
    print("Starting download pool")
    pool = Pool()
    res = pool.map(download_file, url_list)
    print("Jobs sent")
    pool.close()
    pool.join()
    print("Downloads finished")
    print(res)
    return res

def download_and_stack(year, variable):
    dsets_urls = list(map(lambda x: get_year_ensemble(x, variable = variable), [year]))[0]
    filenames = download_file_list(dsets_urls)
    datasets = [ netCDF4.Dataset(filename) for filename in filenames ]
    dask_arrays = []
    for dset in datasets:
        dask_arrays.append(da.from_array(dset[str(variable)], chunks= (366, 144, 144)))
    try:
        final_stack = da.stack(dask_arrays, axis = 0)
    except:
        return filenames, None, None
    return filenames, datasets, final_stack

def get_stacks_mod_avg(a, chunksize):
    nmodels, time, lat, lon = a.shape
    nstacks_lat = int(np.ceil(lat / chunksize))
    nstacks_lon = int(np.ceil(lon / chunksize))
    
    stacks = []
    
    for i in range(nstacks_lat):
        for j in range(nstacks_lon):
            latmin, latmax = i * chunksize, (i+1) * chunksize
            lonmin, lonmax = j * chunksize, (j+1) * chunksize
            print(i, j, '~>', latmin, latmax, lonmin, lonmax)
            stacked = a[:, :, latmin:latmax, lonmin:lonmax]
            print(stacked)
            stacks.append(stacked)
    return stacks

def copy_dataset(src, output_filename):
    dst = netCDF4.Dataset('/home/jovyan/work/' + output_filename, "w")
    # copy attributes
    for name in src.ncattrs():
        dst.setncattr(name, src.getncattr(name))
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(
            name, (len(dimension) if not dimension.isunlimited else None))
    dst.close()
    return output_filename

def restack(chunk_list, chunksize):
    shapes = list(map(np.shape, chunk_list))
    ndays = shapes[0][0]
    nlons = int(1440 / chunksize)
    nlats = int(720 / chunksize)

    out_array = np.empty((ndays, 720, 1440))
 
    combs = list(itertools.product(
        list(range(nlats)),
        list(range(nlons))
    ))
    
    res_list = zip(combs, chunk_list)
    
    for position, arr in res_list:
        minlon, maxlon = position[0] * chunksize, position[0] * chunksize + chunksize
        minlat, maxlat = position[1] * chunksize, position[1] * chunksize + chunksize
        out_array[:, minlon:maxlon, minlat:maxlat] = arr
    return out_array

In [2]:
client.restart()

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787,Cluster Workers: 1 Cores: 8 Memory: 31.62 GB


In [3]:
# Processing pr years
years = list(range(1971, 2001))
var = 'pr'
for i, year in enumerate(years):
    filenames, datasets, current_year_stack = download_and_stack(year, variable=var)
    if current_year_stack is not None:
        chunked_stack = get_stacks_mod_avg(current_year_stack, 360)
        mean = list(map(lambda x: delayed(np.mean)(x, axis=0).compute(), chunked_stack))
        out_array = restack(mean, 360)
        np.save('/home/jovyan/work/' + str(year) + '_pr_avg.npy', out_array)
        del chunked_stack
        del mean
        del out_array
        del current_year_stack
        #output_dataset_filename = copy_dataset(datasets[0], str(year) + '_ensemble_' + var + '.nc')
        #output_dataset = netCDF4.Dataset(output_dataset_filename, 'w')
        #print(output_dataset)
        
    # Finally
    for filename in filenames:
        os.remove(filename)

Starting download pool
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CCSM4_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-CM3_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CNRM-CM5_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_BNU-ESM_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CanESM2_1971.nc
url: http://nasanex.s3.amazonaws.com/NEX-G

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2G_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-LR_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC-ESM-CHEM_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC-ESM_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC5_1972.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP

Jobs sent
Downloads finished
['/temp/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_BNU-ESM_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CCSM4_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CNRM-CM5_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CanESM2_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_GFDL-CM3_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2G_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-LR_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_MIROC-ESM-CHEM_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_MIROC-ESM_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_MIROC5_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_MPI-ESM-LR_1973.nc', '/temp/pr_day_BCSD_historical_r1i1p1_MPI-ESM-MR_1973.

Starting download pool
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1975.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CanESM2_1975.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CNRM-CM5_1975.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-CM3_1975.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1975.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_BNU-ESM_1975.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CCSM4_1975.nc
url: http://nasanex.s3.amazonaws.com/N

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2G_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-LR_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC-ESM-CHEM_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC-ESM_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC5_1976.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP

...

0 0 ~> 0 360 0 360
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 144)>
0 1 ~> 0 360 360 720
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 72)>
0 2 ~> 0 360 720 1080
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 144)>
0 3 ~> 0 360 1080 1440
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 72)>
1 0 ~> 360 720 0 360
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 144)>
1 1 ~> 360 720 360 720
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 72)>
1 2 ~> 360 720 720 1080
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 144)>
1 3 ~> 360 720 1080 1440
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 72)>
Starting download pool
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CCSM4_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2G_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2M_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-LR_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC-ESM-CHEM_1996.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MPI-ESM-MR_1997.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MRI-CGCM3_1997.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_NorESM1-M_1997.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_bcc-csm1-1_1997.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_inmcm4_1997.nc
Jobs sent
Downloads finished
['/temp/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1997.nc', '/temp/pr_day_BCSD_historical_r1i1p1_BNU-ESM_1997.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CCSM4_1997.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1997.nc', '/temp/pr_day_BCSD_historical_r1i1p1_CNRM-CM5_1997.nc', '/temp/pr_day_BCSD

0 0 ~> 0 360 0 360
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 144)>
0 1 ~> 0 360 360 720
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 72)>
0 2 ~> 0 360 720 1080
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 144)>
0 3 ~> 0 360 1080 1440
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 144, 72)>
1 0 ~> 360 720 0 360
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 144)>
1 1 ~> 360 720 360 720
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 72)>
1 2 ~> 360 720 720 1080
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 144)>
1 3 ~> 360 720 1080 1440
dask.array<getitem, shape=(21, 365, 360, 360), dtype=float32, chunksize=(1, 365, 72, 72)>
Starting download pool
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos

url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_CESM1-BGC_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2G_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_GFDL-ESM2M_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-LR_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_IPSL-CM5A-MR_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_MIROC-ESM-CHEM_2000.nc
url: http://nasanex.s3.amazonaws.com/NEX-G

In [4]:
ls -l

total 68000896
-rw-r--r-- 1 jovyan users         96 Nov 21 22:01 1971_ensemble_pr.nc
-rw-r--r-- 1 jovyan users 3027456096 Nov 21 22:55 1971_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 21 23:32 1973_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 00:07 1974_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 00:43 1975_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 01:20 1977_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 01:55 1978_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 02:31 1979_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 03:08 1981_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 03:44 1982_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 04:21 1983_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 04:57 1985_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 05:32 1986_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 06:08 1987_pr_avg.npy
-rw-r--r-- 1 jovyan users 3027456096 Nov 22 06:45 1989_pr